In [32]:
# from google.colab import drive
# drive.mount('/content/drive')


In [33]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
import numpy as np
from sklearn.metrics import accuracy_score

In [34]:
class PositionalEncoder(nn.Module):
    def __init__(self, d_model, max_seq_len=1000):
        super(PositionalEncoder, self).__init__()
        self.d_model = d_model
        self.max_seq_len = max_seq_len

        # Learnable embeddings for positional encoding
        self.pos_embedding = nn.Embedding(max_seq_len, d_model)

    def forward(self, x):
        # Generating a tensor of positional indices (0 to max_seq_len - 1)
        position_indices = torch.arange(0, x.size(1)).unsqueeze(0).repeat(x.size(0), 1).to(x.device) #(32,100)

        # Retrieve the positional encodings corresponding to these indices
        pos_encode = self.pos_embedding(position_indices) #(32,100,128)

        # Add positional encodings to input
        x = x + pos_encode
        return x

In [35]:
# class PositionalEncoder(nn.Module):

#     def __init__(self, d_model, max_sequence_length):
#         super().__init__()
#         self.max_sequence_length = max_sequence_length
#         self.d_model = d_model

#     def forward(self,x):
#         even_i = torch.arange(0, self.d_model, 2).float()
#         denominator = torch.pow(10000, even_i/self.d_model)
#         position = torch.arange(self.max_sequence_length).reshape(self.max_sequence_length, 1)
#         even_PE = torch.sin(position / denominator)
#         odd_PE = torch.cos(position / denominator)
#         stacked = torch.stack([even_PE, odd_PE], dim=2)
#         PE = torch.flatten(stacked, start_dim=1, end_dim=2)
#         PE = PE.unsqueeze(0).expand(x.size(0), -1, -1)
#         x = x + PE
#         return x

In [40]:
#Dummy dataset
vocab_size = 10000
embed_dim = 128
hidden_dim = 256
num_classes = 2
max_seq_len = 100
batch_size = 32

seed = 45
torch.manual_seed(seed)
torch.cuda.manual_seed(seed)
np.random.seed(seed)

train_data = torch.randint(0, vocab_size, (1000, max_seq_len))
train_labels = torch.randint(0, num_classes, (1000,))
test_data = torch.randint(0, vocab_size, (200, max_seq_len))
test_labels = torch.randint(0, num_classes, (200,))

train_dataset = TensorDataset(train_data, train_labels)
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_dataset = TensorDataset(test_data, test_labels)
test_loader = DataLoader(test_dataset, batch_size=batch_size)

class TextClassifier(nn.Module):
    def __init__(self, vocab_size, embed_dim, hidden_dim, num_classes, max_seq_len):
        super(TextClassifier, self).__init__()
        self.embedding = nn.Embedding(vocab_size, embed_dim) # (32,100,128)
        self.positional_encoder = PositionalEncoder(embed_dim, max_seq_len) #(32,100,128)
        self.fc1 = nn.Linear(embed_dim, hidden_dim)
        self.fc2 = nn.Linear(hidden_dim, num_classes) #

    def forward(self, x):
        x = self.embedding(x)  # Shape: [Batch, Seq, Embed]
        #print("input with embedding :", x.shape)#(32,100,128)
        x = self.positional_encoder(x) #(32,100,128)
        x = torch.mean(x, dim=1)  # Average pooling over sequence length #(32,128)
        x = torch.relu(self.fc1(x)) #(32,256)
        x = self.fc2(x) #(32,2)
        return x

In [41]:

#model and optimizer
model = TextClassifier(vocab_size, embed_dim, hidden_dim, num_classes, max_seq_len)
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Training loop
num_epochs = 10
for epoch in range(num_epochs):
    model.train()
    for batch_data, batch_labels in train_loader:
        #print ("batch data ",batch_data.shape)
        optimizer.zero_grad()
        predictions = model(batch_data)
        loss = nn.CrossEntropyLoss()(predictions, batch_labels)
        loss.backward()
        optimizer.step()

# Evaluation
model.eval()
all_predictions = []
all_labels = []
with torch.no_grad():
    for batch_data, batch_labels in test_loader:
        predictions = model(batch_data)
        predicted_labels = torch.argmax(predictions, dim=1)
        all_predictions.extend(predicted_labels.cpu().numpy())
        all_labels.extend(batch_labels.cpu().numpy())

# accuracy as the evaluation metric
accuracy = accuracy_score(all_labels, all_predictions)
print(f"Accuracy: {accuracy * 100:.2f}%")

Accuracy: 57.00%
